In [1]:
import pandas as pd
import os
import urllib

# Download the data

If `wget` is not installed on your machine, you may want to try `curl URL > file.txt` or the `urllib` package in python.

In [13]:
%%bash
mkdir data_ghcn
cd data_ghcn
wget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
wget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2016.csv.gz
gzip -d 2016.csv.gz

mkdir: cannot create directory â€˜data_ghcnâ€™: File exists
--2021-04-19 21:48:07--  https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
Resolving www1.ncdc.noaa.gov (www1.ncdc.noaa.gov)... 205.167.25.177, 205.167.25.171, 205.167.25.168, ...
Connecting to www1.ncdc.noaa.gov (www1.ncdc.noaa.gov)|205.167.25.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10190312 (9.7M) [text/plain]
Saving to: â€˜ghcnd-stations.txt.3â€™

     0K .......... .......... .......... .......... ..........  0%  297K 33s
    50K .......... .......... .......... .......... ..........  1%  591K 25s
   100K .......... .......... .......... .......... ..........  1% 76.9M 17s
   150K .......... .......... .......... .......... ..........  2%  103M 12s
   200K .......... .......... .......... .......... ..........  2%  597K 13s
   250K .......... .......... .......... .......... ..........  3% 86.5M 11s
   300K .......... .......... .......... .......... ..........  3%  132

# Data Preprocessing

In [14]:
def get_vals(line):
    ls = line.split(',')
    station = ls[0]
    time = ls[1]
    val = float(ls[3])
    return [station, time, val]

def get_stations(filename='data_ghcn/ghcnd-stations.txt'):
    df = pd.read_csv(filename, '/t', header=None, engine='python')
    df = df[0].str.split(expand=True)[[0, 1, 2, 3]]
    df.columns = ['Station', 'Latitude', 'Longitude', 'Elevation']
    return df

def process_year(year, stations, col='TAVG', basedir='data_ghcn'):
    tavg = []
    with open(os.path.join(basedir, "%s.csv" % year)) as h:
        l = h.readline()
        while l:
            if col in l:
                v = get_vals(l)
                if v[0] in stations.Station.values:
                    tavg.append(get_vals(l))
            l = h.readline()
    df_tavg = pd.DataFrame(tavg, columns=['Station', 'Date', col])
    df_merged = df_tavg.merge(stations, left_on='Station', right_on='Station', how='left')
    df_merged['Date'] = df_merged['Date'].apply(pd.Timestamp)
    for c in ['Latitude', 'Longitude', col, 'Elevation']:
        df_merged[c] = df_merged[c].astype(float)
    return df_merged[['Station', 'Date', col, 'Latitude', 'Longitude', 'Elevation']]

In [15]:
stations = get_stations()
df1 = process_year('2016', stations, col='TAVG')
stations = stations[stations.Station.isin(df1.Station)]

In [16]:
df2 = process_year('2016', stations, col='PRCP')

In [17]:
df = df1.merge(df2[['Station', 'Date', 'PRCP']], on=['Station', 'Date'])
df.to_csv('../data/daily_global_weather_2016.csv')
df

,Station,Date,TAVG,Latitude,Longitude,Elevation,PRCP
0,AEM00041194,2016-01-01,241.0,25.2550,55.3640,10.4,0.0
1,AG000060390,2016-01-01,121.0,36.7167,3.2500,24.0,0.0
2,AG000060590,2016-01-01,114.0,30.5667,2.8667,397.0,0.0
3,AG000060611,2016-01-01,105.0,28.0500,9.6331,561.0,0.0
4,AGE00147708,2016-01-01,138.0,36.7200,4.0500,222.0,0.0
...,...,...,...,...,...,...,...
1431006,WZ004455110,2016-12-31,259.0,-26.5330,31.3000,641.0,0.0
1431007,ZA000067743,2016-12-31,272.0,-17.8170,25.8170,986.0,0.0
1431008,ZI000067775,2016-12-31,211.0,-17.9170,31.1330,1480.0,20.0
1431009,ZI000067975,2016-12-31,221.0,-20.0670,30.8670,1095.0,109.0


In [7]:
df1

,Station,Date,TAVG,Latitude,Longitude,Elevation
0,AEM00041194,2018-01-01,209.0,25.2550,55.364,10.4
1,AE000041196,2018-01-01,186.0,25.3330,55.517,34.0
2,AEM00041217,2018-01-01,191.0,24.4330,54.651,26.8
3,AEM00041218,2018-01-01,193.0,24.2620,55.609,264.9
4,AFM00040938,2018-01-01,82.0,34.2100,62.228,977.2
...,...,...,...,...,...,...
2331684,WFM00091754,2018-12-31,258.0,-14.3170,-178.117,36.0
2331685,WIM00060096,2018-12-31,192.0,23.7000,-15.867,10.0
2331686,WQW00041606,2018-12-31,258.0,19.2833,166.650,4.3
2331687,WZ004455110,2018-12-31,244.0,-26.5330,31.300,641.0
